In [ ]:
# Notebook 03: Visualising Electrophysiology Data

This notebook demonstrates various visualisations for interpreting in vivo electrophysiology data, including raster plots, peri-stimulus histograms, and time-frequency analysis.

### Objectives:
1. Create raster plots to visualise spike timings across neurons.
2. Generate peri-stimulus time histograms (PSTH) aligned to stimulus events.
3. Plot event-related potentials (ERP) to observe averaged responses.
4. Visualise local field potentials (LFPs) for oscillatory activity.
5. Perform time-frequency analysis using spectrograms.

# Notebook 03: Loading Processed Data
import numpy as np
import matplotlib.pyplot as plt

# Load the spike times and labels from the spike sorting analysis
spike_times = np.load('data/processed/spike_indices.npy')
spike_labels = np.load('data/processed/spike_labels.npy')

# Load the cleaned LFP data
lfp_data = np.load('data/processed/cleaned_data.npy')

# Load additional event information (e.g., stimulus times) if available
stimulus_times = np.load('data/processed/stimulus_times.npy')  # Example path

# Notebook 03: Raster Plot
def plot_raster(spike_times, spike_labels, num_neurons, duration):
    plt.figure(figsize=(12, 6))
    for neuron_id in range(num_neurons):
        neuron_spike_times = spike_times[spike_labels == neuron_id]
        plt.scatter(neuron_spike_times / 20000.0, np.ones_like(neuron_spike_times) * neuron_id, s=1, label=f'Neuron {neuron_id}' if neuron_id < 10 else None)  # Convert samples to seconds
    plt.xlabel('Time (s)')
    plt.ylabel('Neuron ID')
    plt.title('Raster Plot')
    plt.show()

# Generate a raster plot for the first few neurons
plot_raster(spike_times, spike_labels, num_neurons=5, duration=2)  # Adjust `num_neurons` as needed

# Notebook 03: Peri-Stimulus Time Histogram (PSTH)
def compute_psth(spike_times, spike_labels, stimulus_times, window=(-0.1, 0.5), bin_size=0.01, fs=20000):
    bins = np.arange(window[0], window[1] + bin_size, bin_size)
    psth = np.zeros(len(bins) - 1)

    for stim_time in stimulus_times:
        for neuron_id in range(np.max(spike_labels) + 1):
            neuron_spikes = spike_times[(spike_labels == neuron_id) & (spike_times > stim_time + window[0] * fs) & (spike_times < stim_time + window[1] * fs)]
            aligned_spikes = (neuron_spikes - stim_time) / fs  # Convert to time relative to stimulus
            psth += np.histogram(aligned_spikes, bins=bins)[0]

    psth = psth / (len(stimulus_times) * bin_size)  # Normalize by number of stimuli and bin size
    return bins[:-1], psth

# Compute and plot the PSTH
bins, psth = compute_psth(spike_times, spike_labels, stimulus_times)
plt.figure(figsize=(12, 6))
plt.bar(bins, psth, width=0.01)
plt.xlabel('Time (s) relative to stimulus')
plt.ylabel('Spike Rate (Hz)')
plt.title('Peri-Stimulus Time Histogram (PSTH)')
plt.show()

# Notebook 03: Event-Related Potential (ERP) Plot
def compute_erp(lfp_data, stimulus_times, window=(-0.1, 0.5), fs=20000):
    num_samples = int((window[1] - window[0]) * fs)
    erp = np.zeros(num_samples)

    for stim_time in stimulus_times:
        start_idx = int(stim_time + window[0] * fs)
        end_idx = start_idx + num_samples
        erp += lfp_data[start_idx:end_idx]

    erp /= len(stimulus_times)
    time = np.linspace(window[0], window[1], num_samples)
    return time, erp

# Compute and plot the ERP
time, erp = compute_erp(lfp_data, stimulus_times)
plt.figure(figsize=(12, 6))
plt.plot(time, erp)
plt.xlabel('Time (s) relative to stimulus')
plt.ylabel('Potential (µV)')
plt.title('Event-Related Potential (ERP)')
plt.show()

# Notebook 03: # Plot the LFP data for a short segment
plt.figure(figsize=(12, 6))
plt.plot(np.arange(0, 2, 1/20000), lfp_data[:40000])  # Plot the first 2 seconds
plt.xlabel('Time (s)')
plt.ylabel('Potential (µV)')
plt.title('Local Field Potential (LFP)')
plt.show()

# Notebook 03: LFP Visualisation
# Plot the LFP data for a short segment
plt.figure(figsize=(12, 6))
plt.plot(np.arange(0, 2, 1/20000), lfp_data[:40000])  # Plot the first 2 seconds
plt.xlabel('Time (s)')
plt.ylabel('Potential (µV)')
plt.title('Local Field Potential (LFP)')
plt.show()

# Notebook 03: Time-Frequency Analysis (Spectogram)
from scipy.signal import spectrogram

frequencies, times, Sxx = spectrogram(lfp_data, fs=20000, nperseg=1024)

plt.figure(figsize=(12, 6))
plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx), shading='gouraud')
plt.title('Spectrogram (Time-Frequency Analysis)')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.ylim([0, 100])  # Focus on the lower frequency range for LFP analysis
plt.colorbar(label='Power (dB)')
plt.show()